In [ ]:
# install pip and pandas

# !pip install pip --upgrade
# !pip install pandas

In [21]:
# Import the CSV File
from google.colab import files
import pandas as pd

# This will prompt you to upload a file
uploaded = files.upload()

Saving emails_top_2mm_lines.csv to emails_top_2mm_lines (1).csv


In [25]:
# Pull sample data from emails_top_2mm_lines.csv.
main = pd.read_csv("/content/emails_top_2mm_lines.csv")
main.head()

# Note: Full dataset results in an EOF error on line ~32,000

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [26]:
# View Message
main['message'][0]

"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

In [28]:
# Parse attributes of email text based on text delimiters.
def get_message(Series: pd.Series):
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        del message_words[:15]
        result.iloc[row] = ''.join(message_words).strip()
    return result

def get_date(Series: pd.Series):
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        del message_words[0]
        del message_words[1:]
        result.iloc[row] = ''.join(message_words).strip()
        result.iloc[row] = result.iloc[row].replace('Date: ', '')
    print('Done parsing, converting to datetime format..')
    return pd.to_datetime(result)

def get_sender_and_receiver(Series: pd.Series):
    sender = pd.Series(index = Series.index)
    recipient1 = pd.Series(index = Series.index)
    recipient2 = pd.Series(index = Series.index)
    recipient3 = pd.Series(index = Series.index)

    for row,message in enumerate(Series):
        message_words = message.split('\n')
        sender[row] = message_words[2].replace('From: ', '')
        recipient1[row] = message_words[3].replace('To: ', '')
        recipient2[row] = message_words[10].replace('X-cc: ', '')
        recipient3[row] = message_words[11].replace('X-bcc: ', '')

    return sender, recipient1, recipient2, recipient3


def get_subject(Series: pd.Series):
    result = pd.Series(index = Series.index)

    for row, message in enumerate(Series):
        message_words = message.split('\n')
        message_words = message_words[4]
        result[row] = message_words.replace('Subject: ', '')
    return result

def get_folder(Series: pd.Series):
    result = pd.Series(index = Series.index)

    for row, message in enumerate(Series):
        message_words = message.split('\n')
        message_words = message_words[12]
        result[row] = message_words.replace('X-Folder: ', '')
    return result

In [29]:
# Convert the attributes into Columns in table.
main['text'] = get_message(main.message)
main['sender'], main['recipient1'], main['recipient2'], main['recipient3'] = get_sender_and_receiver(main.message)
main['Subject'] = get_subject(main.message)
main['folder'] = get_folder(main.message)
main['date'] = get_date(main.message)

<ipython-input-28-dacbb1bec7a4>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Here is our forecast' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.iloc[row] = ''.join(message_words).strip()
<ipython-input-28-dacbb1bec7a4>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'phillip.allen@enron.com' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sender[row] = message_words[2].replace('From: ', '')
<ipython-input-28-dacbb1bec7a4>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'tim.belden@enron.com' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  recipient1[row] = message_words[3].replace('To: ', '')
<ipyth

Done parsing, converting to datetime format..


<ipython-input-28-dacbb1bec7a4>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(result)
<ipython-input-28-dacbb1bec7a4>:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  return pd.to_datetime(result)


In [30]:
# View cleaned table
main

,file,message,text,sender,recipient1,recipient2,recipient3,Subject,folder,date
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast,phillip.allen@enron.com,tim.belden@enron.com,,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",2001-05-14 16:39:00-07:00
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,phillip.allen@enron.com,john.lavorato@enron.com,,,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",2001-05-04 13:51:00-07:00
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,phillip.allen@enron.com,leah.arsdall@enron.com,,,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,2000-10-18 03:00:00-07:00
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the salar...",phillip.allen@enron.com,randall.gay@enron.com,,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,2000-10-23 06:13:00-07:00
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,phillip.allen@enron.com,greg.piper@enron.com,,,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,2000-08-31 05:07:00-07:00
...,...,...,...,...,...,...,...,...,...,...
31923,beck-s/sent/776.,Message-ID: <12947213.1075856074188.JavaMail.e...,I would like to have my directors and senior d...,sally.beck@enron.com,hector.mcloughlin@enron.com,,,Re: FW: Director-level Impact and Influence,\Sally_Beck_Jun2001\Notes Folders\Sent,2001-03-30 03:50:00-08:00
31924,beck-s/sent/777.,Message-ID: <3329027.1075856074221.JavaMail.ev...,"Good response, Bob. Thanks for leading the wa...",sally.beck@enron.com,robert.superty@enron.com,,,Re: CommodityLogic,\Sally_Beck_Jun2001\Notes Folders\Sent,2001-03-30 03:13:00-08:00
31925,beck-s/sent/778.,Message-ID: <15721034.1075856074242.JavaMail.e...,FYI - See attachment for latest attempt at sat...,sally.beck@enron.com,greg.piper@enron.com,,,AEP-HPL Transition Issues,\Sally_Beck_Jun2001\Notes Folders\Sent,2001-03-29 08:26:00-08:00
31926,beck-s/sent/779.,Message-ID: <15829921.1075856074265.JavaMail.e...,All eight students that we interviewed for the...,sally.beck@enron.com,"billy.lemmons@enron.com, ben.glisan@enron.com",,,UT/Enron MBA Excellence 2001 Scholarship Winne...,\Sally_Beck_Jun2001\Notes Folders\Sent,2001-03-29 05:40:00-08:00


In [32]:
# Return cleaned CSV
# main.to_csv('../data/processed_emails.csv',index=False)
# main.to_csv('../data/processed_emails_top_2mm_lines.csv',index=False)

# Issues accessing my local folder - download through colab.
main.to_csv("/content/processed_emails_top_2mm_lines.csv")
files.download("/content/processed_emails_top_2mm_lines.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>